In [3]:
import gradio as gr
import face_recognition
import os
import numpy as np
from tqdm import tqdm


In [5]:
# Bước 1: Encode khuôn mặt
def encode_faces(folder_path):
    encoded_faces = []
    face_names = []
    image_files = os.listdir(folder_path)  # Lấy danh sách file ảnh
    for file_name in tqdm(image_files, desc="Encoding faces", unit="image"):
        image_path = os.path.join(folder_path, file_name)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image, model="cnn")
        if len(encodings) > 0:
            encoded_faces.append(encodings[0])
            face_names.append(os.path.splitext(file_name)[0])  # Lưu tên từ file
    return encoded_faces, face_names

def recognize_face_with_fit(test_image_path, known_faces, known_names, tolerance=0.5, fit_threshold=50):
    # Load ảnh và đảm bảo chuyển về RGB
    test_image = face_recognition.load_image_file(test_image_path)
    # Sử dụng mô hình "cnn" để phát hiện khuôn mặt
    face_locations = face_recognition.face_locations(test_image, model="cnn")
    test_encodings = face_recognition.face_encodings(test_image, face_locations)
    
    if len(test_encodings) == 0:
        print("Không tìm thấy khuôn mặt trong ảnh kiểm tra.")
        return "Unknown", 0
    
    for test_encoding in test_encodings:
        distances = face_recognition.face_distance(known_faces, test_encoding)
        best_match_index = np.argmin(distances)  # Lấy khuôn mặt có khoảng cách nhỏ nhất
        
        if distances[best_match_index] <= tolerance:
            fit_percentage = (1 - distances[best_match_index]) * 100
            if fit_percentage >= fit_threshold:  # So sánh với ngưỡng
                return known_names[best_match_index], fit_percentage
            else:
                return "Unknown", fit_percentage
    
    return "Unknown", 0

# Bước 3: Thực hiện train
# Encode ảnh trong folder "B21DCCN146"
known_faces, known_names = encode_faces("IMAGE")

# Bước 4: Thực hiện test
# Test với ảnh mới
test_image_path = "IMAGE/t.jpg"
result_name, fit_percentage = recognize_face_with_fit(test_image_path, known_faces, known_names)

# Hiển thị kết quả
if result_name == "Unknown":
    print(f"Kết quả nhận diện: Unknown (Độ khớp: {fit_percentage:.10f}%)")
else:
    print(f"Kết quả nhận diện: {result_name} (Độ khớp: {fit_percentage:.10f}%)")


Encoding faces:   0%|          | 0/2 [00:00<?, ?image/s]

Encoding faces: 100%|██████████| 2/2 [00:10<00:00,  5.06s/image]


Kết quả nhận diện: t (Độ khớp: 93.9323630483%)


In [8]:
import cv2
from matplotlib import pyplot as plt

def show_image_with_detected_faces(image_path):
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_image)
    
    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Detected Faces")
    plt.axis("off")
    plt.show()

# Test với ảnh
show_image_with_detected_faces(test_image_path)


ModuleNotFoundError: No module named 'cv2'

In [9]:
def recognize_face_with_fit(test_image, known_faces, known_names, tolerance=0.5, fit_threshold=50):
    # Ảnh từ Gradio đã là numpy array, không cần load lại
    # Phát hiện khuôn mặt
    face_locations = face_recognition.face_locations(test_image, model="cnn")
    test_encodings = face_recognition.face_encodings(test_image, face_locations)
    
    if len(test_encodings) == 0:
        return "Không tìm thấy khuôn mặt trong ảnh kiểm tra."
    
    for test_encoding in test_encodings:
        distances = face_recognition.face_distance(known_faces, test_encoding)
        best_match_index = np.argmin(distances)  # Lấy khuôn mặt có khoảng cách nhỏ nhất
        
        if distances[best_match_index] <= tolerance:
            fit_percentage = (1 - distances[best_match_index]) * 100
            if fit_percentage >= fit_threshold:  # So sánh với ngưỡng
                return f"Nhận diện thành công: {known_names[best_match_index]} với độ khớp {fit_percentage:.2f}%"
            else:
                return f"Unknown (Độ khớp: {fit_percentage:.2f}%)"
    
    return "Unknown"


In [10]:
import gradio as gr
import numpy as np

# Tạo giao diện với hàm nhận diện đã sửa
interface = gr.Interface(
    fn=lambda img: recognize_face_with_fit(img, known_faces, known_names),  # Gọi hàm nhận diện
    inputs=gr.Image(type="numpy"),  # Input: Tải ảnh dưới dạng numpy array
    outputs="text",  # Output: Kết quả nhận diện
    title="Face Recognition",
    description="Tải lên một ảnh để nhận diện khuôn mặt."
)

# Chạy giao diện
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
